In [ ]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
import datetime
import concurrent

from phantasy import fetch_data, caget

In [ ]:
from maximize_FC import maximize_FC814, maximize_FC977, maximize_FC998, maximize_FC1102
from pyBO import pyBO
from pyBO.model import GaussianProcess
from pyBO.covfunc import matern52
from pyBO.util import proximal_ordered_init_sampler
from pyBO import Auto

In [ ]:
now = str(datetime.datetime.now())
now = now[:now.rfind(':')].replace(' ','_').replace(':',';')
now

# preprare objective

In [ ]:
obj = maximize_FC1102(decision_L2reg=0.01,
                      decision_CSETs=['FE_LEBT:PSC2_D0948:I_CSET', 'FE_LEBT:PSC1_D0948:I_CSET',
                                      'FE_LEBT:PSC2_D0964:I_CSET', 'FE_LEBT:PSC1_D0964:I_CSET',
                                      'FE_LEBT:PSC2_D0979:I_CSET', 'FE_LEBT:PSC1_D0979:I_CSET',],
                      decision_min=[-2.5]*6,
                      decision_max=[ 2.5]*6,
                      weight_BCM_ratio=0.8,
                 )
# obj.decision_bounds = np.array([[-2,2],[-2,2]])

In [ ]:
obj.prepare_objective()

 # BO intial training data sampling

In [ ]:
obj.x0

In [ ]:
n_init = 64

train_x = proximal_ordered_init_sampler(n_init,
                                      bounds=obj.decision_bounds,
                                      x0=obj.x0,
                                      ramping_rate=obj.decision_tols,
                                      polarity_change_time=15
                                     )
train_y = np.zeros(n_init)
for i,x in enumerate(train_x):
    train_y[i] = obj(x) # + 1e-6+np.random.randn()

In [ ]:
fig,ax = plt.subplots(figsize=(3,3))
ax.plot(train_x[:,0],train_x[:,1],alpha=0.5,c='r')
ax.scatter(train_x[:,0],train_x[:,1])
ax.hlines(0,obj.decision_bounds[0,0],obj.decision_bounds[0,1],ls='--',color='k');
ax.vlines(0,obj.decision_bounds[1,0],obj.decision_bounds[1,1],ls='--',color='k');
ax.set_xlim(obj.decision_bounds[0,0],obj.decision_bounds[0,1]);
ax.set_ylim(obj.decision_bounds[1,0],obj.decision_bounds[1,1]);
ax.set_xlabel(obj.decision_CSETs[0]);
ax.set_ylabel(obj.decision_CSETs[1]);
fig.tight_layout()
fig.savefig(now+"_FC1102_init_samples.png",dpi=180)

In [ ]:
model = GaussianProcess(matern52())
bo = pyBO.BO(model=model,
             x=train_x,
             y=train_y,
             bounds=obj.decision_bounds,
             acquisition_func= 'EI',
            )

In [ ]:
x1 = bo.query_candidates(
                         polarity_penalty=0.1,
                         X_penal = None, 
                         X_favor = None)

executor = concurrent.futures.ThreadPoolExecutor(max_workers=1)
y1_future = executor.submit(obj,x1.flatten())

In [ ]:
x2 = bo.query_candidates(
                         X_current = Auto,
                         x_candidate=x1,
                         y_candidate_future=y1_future,
                         polarity_penalty=0.1,
                         X_penal = Auto, 
                         X_favor = None)

In [ ]:
y1 = [y1_future.result()]
y2_future = executor.submit(obj,x2.flatten())
bo.update_model(x1=x1,y1=y1)

In [ ]:
x = x2
y_future = y2_future
# for i in range(32):
for i in range(64-2):
    print(i)
    x,y_future = bo.step(obj, x, y_future, polarity_penalty=Auto, X_penal=Auto, X_favor=None)

In [ ]:
fig,ax = plt.subplots(figsize=(5,3))
bo.plot_obj_history(ax=ax)
fig.tight_layout()
fig.savefig(now+"_FC1102_history.png",dpi=180)

In [ ]:
best_x,best_y, = bo.best_sofar()
best_x,best_y

In [ ]:
obj(best_x)

In [ ]:
# obj(x.flatten())

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,8))
bo.plot_acquisition_2D_projection(epoch=0, fig=fig, ax=ax[0,0], project_maximum=True, grid_ponits_each_dim=16)
ax[0,0].set_title('epoch 0');
bo.plot_acquisition_2D_projection(epoch=1, fig=fig, ax=ax[0,1], project_maximum=True, grid_ponits_each_dim=16)
ax[0,1].set_title('epoch 1');
bo.plot_acquisition_2D_projection(epoch=2, fig=fig, ax=ax[1,0], project_maximum=True, grid_ponits_each_dim=16)
ax[1,0].set_title('epoch 2');
bo.plot_acquisition_2D_projection(epoch=-1, fig=fig, ax=ax[1,1], project_maximum=True, grid_ponits_each_dim=16)
ax[1,1].set_title('Final epoch');

ax[1,0].set_xlabel(obj.decision_CSETs[0])
ax[1,1].set_xlabel(obj.decision_CSETs[0])
ax[0,0].set_ylabel(obj.decision_CSETs[1])
ax[0,1].set_ylabel(obj.decision_CSETs[1])

fig.tight_layout()
fig.savefig(now+"_FC1102_acquisition.png",dpi=180)

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,8))
bo.plot_model_2D_projection(epoch=0, fig=fig, ax=ax[0,0], project_maximum=True, grid_ponits_each_dim=16)
ax[0,0].set_title('epoch 0');
bo.plot_model_2D_projection(epoch=1, fig=fig, ax=ax[0,1], project_maximum=True, grid_ponits_each_dim=16)
ax[0,1].set_title('epoch 1');
bo.plot_model_2D_projection(epoch=2, fig=fig, ax=ax[1,0], project_maximum=True, grid_ponits_each_dim=16)
ax[1,0].set_title('epoch 2');
bo.plot_model_2D_projection(epoch=-1, fig=fig, ax=ax[1,1], project_maximum=True, grid_ponits_each_dim=16)
ax[1,1].set_title('Final epoch');

ax[1,0].set_xlabel(obj.decision_CSETs[0])
ax[1,1].set_xlabel(obj.decision_CSETs[0])
ax[0,0].set_ylabel(obj.decision_CSETs[1])
ax[0,1].set_ylabel(obj.decision_CSETs[1])

fig.tight_layout()
fig.savefig(now+"_FC1102_mean_model.png",dpi=180)

In [ ]:
best_x, best_y

In [ ]:
now